<h1><center>OPTUNA vs БАЙЕС</center></h1>

Вернёмся к анализу датасета «Космолайна» об оттоке клиентов. До этого вы применяли к нему методы Grid Search и Random Search для оптимизации гиперпараметров модели. Новая задача — использовать байесовский подход. Вы будете работать с алгоритмом TPE. 

Как и до этого, вам предстоит обучить новую версию модели — но в этот раз интегрировав процесс обучения с библиотекой optuna для оптимизации гиперпараметров. Так, вы не только автоматизируете поиск наилучших настроек для модели, но и сделаете его более точным и целенаправленным. 

Вы также потренируетесь проводить интеграцию с MLflow. Напомним, что это позволяет систематизировать и сохранять информацию обо всех экспериментах, проведённых в ходе подбора гиперпараметров. И к тому же поможет анализировать результаты.

Не забудем и про воспроизводимость. Для этого вы сохраните всю информацию о процессе подбора гиперпараметров в локальную базу данных. В реальных проектах это будет отличной страховкой, ведь вы сможете в любой момент восстановить условия эксперимента, повторить его и проверить результаты.

Так начнём же.

---

Импортируем библиотеки и настроим параметры:

In [ ]:
import pandas as pd
import numpy as np
import os
import psycopg
import mlflow
from mlflow.tracking import MlflowClient
from catboost import CatBoostClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import ( OneHotEncoder, SplineTransformer, QuantileTransformer, StandardScaler,
                                    RobustScaler, PolynomialFeatures, KBinsDiscretizer )
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autofeat import AutoFeatRegressor, AutoFeatClassifier

import catboost as cb
from catboost import CatBoostClassifier
from category_encoders import CatBoostEncoder

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Lasso
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

import optuna
from optuna.integration.mlflow import MLflowCallback
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from optuna.samplers import CmaEsSampler
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from collections import defaultdict


TABLE_NAME = "clean_users_churn" # таблица с данными
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# EXPERIMENT_NAME = "model_bayesian_search" # ваш код здесь
# RUN_NAME = "model_bayesian_search_run"
# REGISTRY_MODEL_NAME = 'model_bayesian_search' # ваш код здесь
# MLFLOW_PARENT_RUN_ID = 1 #'model_bayesian_search'

# STUDY_DB_NAME = "sqlite:///local.study.db"
# STUDY_NAME = "model_bayesian_search"

# experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)\
#     if not mlflow.get_experiment_by_name(EXPERIMENT_NAME)\
#     else mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

Загрузим таблицу:

In [ ]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {"host": 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net', #os.getenv("DB_DESTINATION_HOST"),
                        "port": '6432', #os.getenv("DB_DESTINATION_PORT"),
                        "dbname": 'playground_mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_NAME"),
                        "user": 'mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_USER"),
                        "password": '0c567edd8ad8472e87d5c85cc4d664e4' } #os.getenv("DB_DESTINATION_PASSWORD")}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df['target'] = (df['end_date'].notna()).astype(int)
df.head(2) 

Подготовим данные для обучения:

In [ ]:
obj_df = df.select_dtypes(include="object")

**Задание 1**

После того как вы выделили категориальные колонки, закодируйте их для подачи в вашу модель:

In [ ]:
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first') # ваш код здесь #

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy()) # ваш код здесь #

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out(cat_columns)) # ваш код здесь #

# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)

Сейчас поработайте с числовыми признаками: monthly_charges и total_charges. Из них можно сгенерировать довольно много признаков для вашей модели. 

**Задание 2**

Напишите код преобразования числовых признаков в списке num_columns, используя следующие энкодеры:
- SplineTransformer,
- QuantileTransformer,
- RobustScaler,
- PolynomialFeatures,
- KBinsDiscretizer.

In [ ]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

# num_df = df.select_dtypes(include=['number'])

num_df = df[num_columns].copy()

# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline) # ваш код здесь #
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame( encoded_features, columns=encoder_spl.get_feature_names_out(num_columns) )
num_df = pd.concat([num_df, encoded_df], axis=1)

# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles) #, output_distribution='normal') # ваш код здесь #
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# RobustScaler
encoder_rb = RobustScaler() # ваш код здесь #
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree) # ваш код здесь #
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns)) # ваш код здесь #
# get all columns after the intercept and original features
encoded_df.columns = encoder_pol.get_feature_names_out(num_columns)
encoded_df = encoded_df.iloc[:, 1 + len(num_columns):]
encoded_df.columns = [f"{col}_poly" for col in encoded_df.columns]

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample) # ваш код здесь #
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1) # ваш код здесь #

num_df.head(2)

Трансформируем данные:

In [ ]:
numeric_transformer = ColumnTransformer(transformers=[('spl', encoder_spl, num_columns),('q', encoder_q, num_columns), ('rb', encoder_rb, num_columns), ('pol', encoder_pol, num_columns), ('kbd', encoder_kbd, num_columns)])
categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)] )
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_columns), ('cat', categorical_transformer, cat_columns)], n_jobs=-1)
encoded_features = preprocessor.fit_transform(df) # ваш код здесь #
transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out()) # ваш код здесь #

df = pd.concat([df, transformed_df], axis=1) # ваш код здесь #
df.head(2)

Выберем закодированные данные:

In [ ]:
df = df[['target', 'num__spl__monthly_charges_sp_0',
       'num__spl__monthly_charges_sp_1', 'num__spl__monthly_charges_sp_2',
       'num__spl__monthly_charges_sp_3', 'num__spl__monthly_charges_sp_4',
       'num__spl__monthly_charges_sp_5', 'num__spl__total_charges_sp_0',
       'num__spl__total_charges_sp_1', 'num__spl__total_charges_sp_2',
       'num__spl__total_charges_sp_3', 'num__spl__total_charges_sp_4',
       'num__spl__total_charges_sp_5', 'num__q__monthly_charges',
       'num__q__total_charges', 'num__rb__monthly_charges',
       'num__rb__total_charges', 'num__pol__1', 'num__pol__monthly_charges',
       'num__pol__total_charges', 'num__pol__monthly_charges^2',
       'num__pol__monthly_charges total_charges', 'num__pol__total_charges^2',
       'num__pol__monthly_charges^3',
       'num__pol__monthly_charges^2 total_charges',
       'num__pol__monthly_charges total_charges^2',
       'num__pol__total_charges^3', 'num__kbd__monthly_charges',
       'num__kbd__total_charges', 'cat__type_One year', 'cat__type_Two year',
       'cat__payment_method_Credit card (automatic)',
       'cat__payment_method_Electronic check',
       'cat__payment_method_Mailed check', 'cat__internet_service_Fiber optic',
       'cat__gender_Male']]

# Разделение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

Применим optuna:

In [ ]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"
os.environ['MLFLOW_ARTIFACT_URI'] = 'http://s3-student-mle-20250529-05fed48463'

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

EXPERIMENT_NAME = "model_bayesian_search" # ваш код здесь
RUN_NAME = "model_bayesian_search"
# MLFLOW_PARENT_RUN_ID = "model_bayesian_search_parent"

STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

optuna.logging.set_verbosity(optuna.logging.INFO)

# def objective(trial: optuna.Trial) -> float:
def objective(trial, X_train, y_train):
    param = { "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
              "depth": trial.suggest_int("depth", 1, 12),
              "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
              "random_strength": trial.suggest_float("random_strength", 0.1, 5),
              "loss_function": "Logloss",
              "task_type": "CPU",
              "random_seed": 0,
              "iterations": 300,
              "verbose": False } # ваш код здесь #

    model = CatBoostClassifier(**param)
    skf = StratifiedKFold(n_splits=2) # ваш код здесь #)

    metrics = defaultdict(list)
    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
        # ваш код здесь #
        train_x = X_train.iloc[train_index]  # Добавление train_x
        train_y = y_train.iloc[train_index]
        val_x = X_train.iloc[val_index] # Добавление val_x 
        val_y = y_train.iloc[val_index]
        # X_train_fold, X_val_fold = X_train[train_index], X_train[val_index] # Отключен по совету НС
        # y_train_fold, y_val_fold = y_train[train_index], y_train[val_index] # Отклюсен по совету НС

        X_train_fold, X_val_fold = train_x, val_x
        y_train_fold, y_val_fold = train_y, val_y
        
        model.fit(train_x, train_y)  # Использование train_x для обучения модели
        probas = model.predict_proba(val_x)[:, 1]
        prediction = model.predict(val_x)

        _, err1, _, err2 = confusion_matrix(val_y, prediction, normalize='all').ravel()
        auc = roc_auc_score(val_y, probas)
        precision = precision_score(val_y, prediction)
        recall = recall_score(val_y, prediction)
        f1 = f1_score(val_y, prediction)
        logloss = log_loss(val_y, prediction)
        
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["auc"].append(auc)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["f1"].append(f1)
        metrics["logloss"].append(logloss)

    # ваш код здесь #
    err1 = sum(metrics["err1"]) / len(metrics["err1"])
    err_1 = np.median(np.array(metrics['err1']))
    err2 = sum(metrics["err2"]) / len(metrics["err2"])
    err_2 = np.median(np.array(metrics['err2']))
    auc = np.median(np.array(metrics['auc']))
    precision = np.median(np.array(metrics['precision']))
    recall = np.median(np.array(metrics['recall']))
    f1 = np.median(np.array(metrics['f1']))
    logloss = np.median(np.array(metrics['logloss']))

    return auc

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
#     run_id = run.info.run_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)

    # Сохраняем модель в MLFlow
    mlflow.catboost.log_model(best_model, "best_model", artifact_path="cv")
    mlflow.log_params(best_params)  # записываем параметры модели
    mlflow.log_metrics({"auc": auc, "precision": precision, "recall": recall, "f1": f1, "logloss": logloss})

mlflc = MLflowCallback(metric_name='AUC', tracking_uri=f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}',
                       create_experiment=False,
                       mlflow_kwargs={'experiment_id': experiment_id, 'tags':{MLFLOW_PARENT_RUN_ID:run_id}}) # ваш код здесь #

study = optuna.create_study(direction='maximize', study_name=STUDY_NAME, storage=STUDY_DB_NAME,\
                            sampler=optuna.samplers.TPESampler(), load_if_exists=True) # ваш код здесь #
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=10, callbacks=[mlflc])
best_params = study.best_params # ваш код здесь #

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best params: {best_params}")

In [ ]:
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    model = CatBoostClassifier(**best_params)
    model.fit(X_train, y_train)
    mlflow.catboost.log_model(model, "best_model")  # сохраняем модель в MLFlow
    # mlflow.log_params(param)  # записываем параметры модели
    mlflow.log_metrics({"auc": auc, "precision": precision, "recall": recall, "f1": f1, "logloss": logloss})